In [ ]:
!pip install requests
!pip install beautifulsoup4
!pip install wikipedia-api
!pip install tqdm
!pip install random

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15384 sha256=d5fa536af686595283861b606c6ad2013dac79b4c34d9399acc6aa2bd005e043
  Stored in directory: /root/.cache/pip/wheels/0b/0f/39/e8214ec038ccd5aeb8c82b957289f2f3ab2251febeae5c2860
Successfully built wikipedia-api
ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests
from bs4 import BeautifulSoup
import wikipediaapi
import json
from tqdm import tqdm
import time
import random
from urllib.parse import unquote

In [ ]:
def get_creation_date(page_url):
    """
    This function uses the Wikipedia API to fetch the revision history of a page
    and checks if the creation date is in August 2023.
    """
    page_title = unquote(page_url.split('/')[-1])

    api_url = f"https://en.wikipedia.org/w/api.php"
    params = {
        'action': 'query',
        'titles': page_title,
        'prop': 'revisions',
        'rvlimit': 1,
        'rvdir': 'newer',
        'format': 'json',
    }

    response = requests.get(api_url, params=params)
    data = response.json()

    pages = data.get('query', {}).get('pages', {})
    for page_id, page_data in pages.items():
        if 'revisions' in page_data:
            creation_timestamp = page_data['revisions'][0]['timestamp']
            creation_date = creation_timestamp[:7]
            if creation_date == '2023-08':
                print(f"Article '{page_title}' created on {creation_date}")
                return True

    return False


def scrape_wikipedia_all_pages(limit=2000):
    base_url = "https://en.wikipedia.org/w/index.php?title=Special:AllPages&from=%22Ako+ang+Batas%22+-Gen.+Tomas+Karingal"
    next_page = None
    articles = []
    page_count = 0

    while len(articles) < limit:
        url = base_url if not next_page else f"https://en.wikipedia.org{next_page}"
        print(f"Fetching {url}")
        headers = {"User-Agent": "Mozilla/5.0"}

        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
        except requests.RequestException as e:
            print(f"Error fetching {url}: {e}")
            time.sleep(5)
            continue

        soup = BeautifulSoup(response.text, "html.parser")

        for link in soup.select(".mw-allpages-chunk a"):
            title = link.text
            article_url = f"https://en.wikipedia.org{link['href']}"

            if get_creation_date(article_url):
                articles.append((title, article_url))
                if len(articles) >= limit:
                    break

        next_link = soup.find("a", string=lambda text: text and text.startswith("Next page"))
        if next_link:
            next_page = next_link["href"]
            print(f"Next page found: {next_page}")
        else:
            print("No more pages to scrape.")
            break

        page_count += 1
        print(f"✅ Scraped {len(articles)} articles so far... (Page {page_count})")

        time.sleep(random.uniform(1, 3))

    print(f"✅ Finished scraping {len(articles)} articles.")
    return articles


In [ ]:
def scrape_wikipedia_article(page_title, wiki_wiki, max_retries=3):
    for attempt in range(max_retries):
        try:
            page = wiki_wiki.page(page_title)
            if not page.exists():
                print(f"Page '{page_title}' does not exist.")
                return None
            return page.text
        except (requests.RequestException, requests.Timeout) as e:
            if attempt < max_retries - 1:
                print(f"Error scraping '{page_title}'. Retrying... (Attempt {attempt + 1}/{max_retries})")
                time.sleep(random.uniform(2, 5))
            else:
                print(f"Failed to scrape '{page_title}' after {max_retries} attempts.")
                return None

In [ ]:
article_limit = 1000
file_path = '/content/drive/My Drive/CSE 517 Project/Colabs/Scraping/Wikipedia/scraped_wiki_articles_8_2023.json'

In [ ]:
print(f"Starting to collect {article_limit} recent articles...")
articles = scrape_wikipedia_all_pages(article_limit)
print(f"Collected {len(articles)} articles.")
data = []

wiki_wiki = wikipediaapi.Wikipedia(
    user_agent='WikipediaScraper (your_email@example.com)',
    language='en',
    timeout=30
)

for article_title, article_url in tqdm(articles, desc="Scraping articles"):
    content = scrape_wikipedia_article(article_title, wiki_wiki)
    if content:
        data.append({
            'title': article_title,
            'url': article_url,
            'content': content
        })
    time.sleep(random.uniform(0.5, 1.5))

with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print(f"Scraped data has been saved to {file_path}")


Starting to collect 2 recent articles...
Fetching https://en.wikipedia.org/w/index.php?title=Special:AllPages&from=%22Ako+ang+Batas%22+-Gen.+Tomas+Karingal
Article '"Casual"_(Chappell_Roan_song)' created on 2024-08
Article '"Checkers"_Speech' created on 2024-08
Next page found: /w/index.php?title=Special:AllPages&from=%22Chisinau%22+airport
✅ Scraped 2 articles so far... (Page 1)
✅ Finished scraping 2 articles.
Collected 2 articles.


Scraping articles: 100%|██████████| 2/2 [00:02<00:00,  1.33s/it]

Scraped data has been saved to /content/drive/My Drive/CSE 517 Project/Colabs/Scraping/Wikipedia/scraped_wiki_articles_TEST.json
